# Gaze data from LSL

In [ ]:
import cv2
import pyxdf
import numpy as np

# Load XDF file 
xdf_file = 'tobii_test_trial_lsl.xdf'
streams, header = pyxdf.load_xdf(xdf_file)

# Load Tobii gaze coordinates
gaze_stream = None
for stream in streams:
    if stream['info']['name'][0] == 'Glasses3_Gaze':
        gaze_stream = stream
        break

if gaze_stream is None:
    raise ValueError("Glasses3_Gaze stream not found in XDF file.")

data = gaze_stream['time_series']
timestamps = gaze_stream['time_stamps']

# Gaze data: [local_ts, gaze_ts, pixel_x, pixel_y, norm_x, norm_y]
local_ts = data[:, 0]
gaze_x = data[:, 4]
gaze_y = data[:, 5]

# Load video
video_path = 'scenevideo.mp4'
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
frame_time = 1.0 / fps 
cv2.namedWindow('Gaze Overlay', cv2.WINDOW_NORMAL)
cv2.resizeWindow('Gaze Overlay', 960, 540)

# Play video with gaze overlay
timestamps = timestamps - timestamps[0]
gazestartfirst = False
if gazestartfirst:
    offset = 8.0
    valid_indices = timestamps >= offset
    timestamps = timestamps[valid_indices] - offset
    local_ts = local_ts[valid_indices]
    gaze_x = gaze_x[valid_indices]
    gaze_y = gaze_y[valid_indices]

frame_idx = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break 
    
    current_video_time = frame_idx * frame_time
    print(current_video_time)

    # Find the nearest gaze
    gaze_index = np.argmin(np.abs(timestamps - current_video_time))
    gx, gy = gaze_x[gaze_index], gaze_y[gaze_index]
    lsl_time = local_ts[gaze_index]
    
    # Convert gaze coordinates to pixel coordinates
    h, w, _ = frame.shape
    px = int(gx * w)
    py = int(gy * h)

    # Draw gaze overlay
    cv2.circle(frame, (px, py), 20, (0, 0, 255), 2)
    
    # Display LSL time on frame
    cv2.putText(frame, f'LSL Time: {lsl_time:.6f}', (10, 30), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    # Show frame
    cv2.imshow('Gaze Overlay', frame)
    if cv2.waitKey(int(frame_time * 1000)) & 0xFF == ord('q'):
        break

    frame_idx += 1

    # Check if window was closed manually (clicking 'X')
    try:
        if cv2.getWindowProperty('Gaze Overlay', cv2.WND_PROP_VISIBLE) < 1:
            break
    except cv2.error:
        # Window was destroyed
        break
    except cv2.error:
        # Window was destroyed
        break

cap.release()
cv2.waitKey(500)
cv2.destroyAllWindows()


# Gaze data from Tobii

In [ ]:
import cv2
import json
import numpy as np

# Load gaze data directly from Tobii
file_path = "gazedata"

timestamps = []
gaze2d_x = []
gaze2d_y = []

with open(file_path, "r") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue  
        try:
            obj = json.loads(line)
            if obj.get("type") == "gaze":
                gaze2d = obj["data"].get("gaze2d", [])
                if gaze2d and len(gaze2d) == 2:
                    timestamps.append(obj["timestamp"])
                    gaze2d_x.append(gaze2d[0])
                    gaze2d_y.append(gaze2d[1])
                else:
                    continue
        except json.JSONDecodeError as e:
            print("Skipping line due to JSON error:", e)

timestamps = np.array(timestamps, dtype=float)
gaze2d_x = np.array(gaze2d_x, dtype=float)
gaze2d_y = np.array(gaze2d_y, dtype=float)

# Load video
video_path = 'scenevideo.mp4'
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
frame_time = 1.0 / fps 
cv2.namedWindow('Gaze Overlay', cv2.WINDOW_NORMAL)
cv2.resizeWindow('Gaze Overlay', 960, 540)

# Play video with gaze overlay
frame_idx = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break 
    
    current_video_time = frame_idx * frame_time

    # Find the nearest gaze
    gaze_index = np.argmin(np.abs(timestamps - current_video_time))
    gx, gy = gaze2d_x[gaze_index], gaze2d_y[gaze_index]
    
    # Convert gaze coordinates to pixel coordinates
    h, w, _ = frame.shape
    px = int(gx * w)
    py = int(gy * h)

    # Draw gaze overlay
    cv2.circle(frame, (px, py), 20, (0, 0, 255), 2)

    # Show frame
    cv2.imshow('Gaze Overlay', frame)
    if cv2.waitKey(int(frame_time * 1000)) & 0xFF == ord('q'):
        break

    frame_idx += 1

    # Check if window was closed manually (clicking 'X')
    try:
        if cv2.getWindowProperty('Gaze Overlay', cv2.WND_PROP_VISIBLE) < 1:
            break
    except cv2.error:
        # Window was destroyed
        break
    except cv2.error:
        # Window was destroyed
        break

cap.release()
cv2.waitKey(500)
cv2.destroyAllWindows()

# Gaze Data LSL and Tobii Comparison

Load data from LSL file (.xdf) AND Tobii file (.gazedata)
Load video
Display eye tracking gaze for both LSL file and Tobii file simultaneously.

Note: To sync up with the video and Tobii gaze data, LSL file .xdf should begin at LSL timestamp 232200.

In [ ]:
import cv2
import pyxdf
import json
import numpy as np
import time
from threading import Thread
from queue import Queue

# Load XDF file for LSL gaze data
xdf_file = 'tobii_test_trial_lsl.xdf'
streams, header = pyxdf.load_xdf(xdf_file)

gaze_stream = None
for stream in streams:
    if stream['info']['name'][0] == 'Glasses3_Gaze':
        gaze_stream = stream
        break

if gaze_stream is None:
    raise ValueError("Glasses3_Gaze stream not found in XDF file.")

lsl_data = gaze_stream['time_series']

# LSL Gaze data: [local_ts, gaze_ts, pixel_x, pixel_y, norm_x, norm_y]
lsl_local_ts = lsl_data[:, 0]
lsl_gaze_x = lsl_data[:, 4]
lsl_gaze_y = lsl_data[:, 5]

# Load Tobii gaze data
file_path = "gazedata"
tobii_timestamps = []
tobii_gaze_x = []
tobii_gaze_y = []

with open(file_path, "r") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue  
        try:
            obj = json.loads(line)
            if obj.get("type") == "gaze":
                gaze2d = obj["data"].get("gaze2d", [])
                if gaze2d and len(gaze2d) == 2:
                    tobii_timestamps.append(obj["timestamp"])
                    tobii_gaze_x.append(gaze2d[0])
                    tobii_gaze_y.append(gaze2d[1])
        except json.JSONDecodeError as e:
            print("Skipping line due to JSON error:", e)

tobii_timestamps = np.array(tobii_timestamps, dtype=float)
tobii_gaze_x = np.array(tobii_gaze_x, dtype=float)
tobii_gaze_y = np.array(tobii_gaze_y, dtype=float)

# Load video
video_path = 'scenevideo.mp4'
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
frame_time = 1.0 / fps

# Sync LSL data (start at LSL timestamp 232200)
lsl_sync_offset = 232200.0

# Setup display queue and control flags
display_queue = Queue(maxsize=2)
quit_flag = False
window_name = 'LSL vs Tobii Comparison'

def display_thread_worker():
    """Worker thread that handles OpenCV display operations."""
    global quit_flag
    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL | cv2.WINDOW_KEEPRATIO)
    cv2.resizeWindow(window_name, 960, 540)
    
    while not quit_flag:
        try:
            # Get the latest frame from the queue (non-blocking)
            if not display_queue.empty():
                frame = display_queue.get_nowait()
                cv2.imshow(window_name, frame)
            
            # Check for key presses and window events
            key = cv2.waitKey(1) & 0xFF
            if key == ord("q"):
                quit_flag = True
                break
            
            # Check if window was closed manually
            try:
                if cv2.getWindowProperty(window_name, cv2.WND_PROP_VISIBLE) < 1:
                    quit_flag = True
                    break
            except cv2.error:
                quit_flag = True
                break
                
        except Exception as e:
            if not quit_flag:
                print(f"Display thread error: {e}")
            break
    
    cv2.destroyWindow(window_name)

# Start display thread
display_thread = Thread(target=display_thread_worker, daemon=True)
display_thread.start()
print("Display thread started - video playback will continue even when dragging the window")

# Main processing loop with proper frame timing
frame_idx = 0
start_time = time.time()

while not quit_flag:
    ret, frame = cap.read()
    if not ret:
        break 
    
    current_video_time = frame_idx * frame_time
    current_lsl_time = lsl_sync_offset + current_video_time
    h, w, _ = frame.shape
    
    # Find nearest LSL gaze using lsl_local_ts
    lsl_index = np.argmin(np.abs(lsl_local_ts - current_lsl_time))
    lsl_gx, lsl_gy = lsl_gaze_x[lsl_index], lsl_gaze_y[lsl_index]
    lsl_time = lsl_local_ts[lsl_index]
    lsl_px = int(lsl_gx * w)
    lsl_py = int(lsl_gy * h)
    
    # Find nearest Tobii gaze
    tobii_index = np.argmin(np.abs(tobii_timestamps - current_video_time))
    tobii_gx, tobii_gy = tobii_gaze_x[tobii_index], tobii_gaze_y[tobii_index]
    tobii_time = tobii_timestamps[tobii_index]
    tobii_px = int(tobii_gx * w)
    tobii_py = int(tobii_gy * h)
    
    # Draw both gaze overlays on a copy of the frame
    display_frame = frame.copy()
    cv2.circle(display_frame, (lsl_px, lsl_py), 20, (0, 0, 255), 2)  # Red for LSL
    cv2.circle(display_frame, (tobii_px, tobii_py), 20, (255, 0, 0), 2)  # Blue for Tobii
    
    # Add labels
    cv2.putText(display_frame, 'LSL (Red)', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    cv2.putText(display_frame, f'LSL Time: {lsl_time:.2f}', (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
    cv2.putText(display_frame, f'LSL Gaze: ({lsl_gx:.3f}, {lsl_gy:.3f})', (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
    cv2.putText(display_frame, 'Tobii (Blue)', (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
    cv2.putText(display_frame, f'Tobii Time: {tobii_time:.2f}', (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
    cv2.putText(display_frame, f'Tobii Gaze: ({tobii_gx:.3f}, {tobii_gy:.3f})', (10, 180), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
    
    # Push to display queue (drop old frames if full)
    if display_queue.full():
        try:
            display_queue.get_nowait()
        except:
            pass
    display_queue.put_nowait(display_frame)
    
    frame_idx += 1
    
    # Maintain proper playback speed
    expected_time = start_time + (frame_idx * frame_time)
    current_time = time.time()
    sleep_time = expected_time - current_time
    if sleep_time > 0:
        time.sleep(sleep_time)

# Cleanup
cap.release()
quit_flag = True
display_thread.join(timeout=2.0)
cv2.destroyAllWindows()
print("Video playback complete")